# _Dataset Creation:_ 
In this section I read in and cleaned customer order and online data for 2016 and 2017 because that's when they both are populated. Then calculated the total counts of online engagement and order history for each of the customers. I then exported this dataset to a pickle file to be used in the next section.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 1000)
pd.set_option('max_columns',100)

In [2]:
from dataset_creation import (
    get_order_total_at_order_level, get_previous_orders_by_month, 
    get_previous_online_sessions_by_week, get_online_sessions_df
)

# Get Order Data

In [3]:
order = pd.read_csv('data/order.csv')

#### Aggregate purchase counts to order level

In [4]:
order_totals = get_order_total_at_order_level(order)

#### Filtering to orders that occured durint the time where we have online data

In [5]:
order_totals = order_totals[
    (order_totals['orderdate']>= pd.to_datetime('2016-01-01'))
    & (order_totals['orderdate']< pd.to_datetime('2018-01-01'))
].copy()

In [6]:
order_totals.shape

(102915, 263)

In [7]:
order_totals.head()

,ordno,custno,P1:1.0,P1:2.0,P1:3.0,P1:4.0,P1:5.0,P1:7.0,P2:-1.0,P2:-7.0,P2:10.0,P2:100.0,P2:101.0,P2:102.0,P2:103.0,P2:104.0,P2:105.0,P2:106.0,P2:107.0,P2:108.0,P2:109.0,P2:11.0,P2:110.0,P2:111.0,P2:112.0,P2:113.0,P2:114.0,P2:115.0,P2:116.0,P2:117.0,P2:118.0,P2:119.0,P2:12.0,P2:120.0,P2:121.0,P2:122.0,P2:123.0,P2:124.0,P2:125.0,P2:126.0,P2:127.0,P2:128.0,P2:129.0,P2:13.0,P2:130.0,P2:131.0,P2:132.0,P2:133.0,P2:134.0,P2:135.0,...,P2:53.0,P2:54.0,P2:55.0,P2:56.0,P2:57.0,P2:58.0,P2:59.0,P2:6.0,P2:60.0,P2:61.0,P2:62.0,P2:63.0,P2:64.0,P2:65.0,P2:66.0,P2:67.0,P2:69.0,P2:7.0,P2:70.0,P2:71.0,P2:72.0,P2:73.0,P2:74.0,P2:75.0,P2:76.0,P2:77.0,P2:78.0,P2:79.0,P2:8.0,P2:80.0,P2:81.0,P2:82.0,P2:83.0,P2:85.0,P2:86.0,P2:88.0,P2:89.0,P2:9.0,P2:90.0,P2:91.0,P2:92.0,P2:93.0,P2:94.0,P2:95.0,P2:96.0,P2:97.0,P2:98.0,P2:99.0,orderdate,ordermonth
0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2017-06-12 08:27:59,6
1,2,2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-03-29 13:04:27,3
2,3,3,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2017-01-31 19:16:49,1
3,4,4,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2017-08-14 21:22:10,8
4,5,5,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016-01-16 16:59:06,1


In [23]:
total_customer_orders =order_totals.groupby('custno', as_index=False)[[
    'P1:1.0', 'P1:2.0', 'P1:3.0', 'P1:4.0',
    'P1:5.0', 'P1:7.0', 'P2:-1.0', 'P2:-7.0', 'P2:10.0', 'P2:100.0',
    'P2:101.0', 'P2:102.0', 'P2:103.0', 'P2:104.0', 'P2:105.0',
    'P2:106.0', 'P2:107.0', 'P2:108.0', 'P2:109.0', 'P2:11.0',
    'P2:110.0', 'P2:111.0', 'P2:112.0', 'P2:113.0', 'P2:114.0',
    'P2:115.0', 'P2:116.0', 'P2:117.0', 'P2:118.0', 'P2:119.0',
    'P2:12.0', 'P2:120.0', 'P2:121.0', 'P2:122.0', 'P2:123.0',
    'P2:124.0', 'P2:125.0', 'P2:126.0', 'P2:127.0', 'P2:128.0',
    'P2:129.0', 'P2:13.0', 'P2:130.0', 'P2:131.0', 'P2:132.0',
    'P2:133.0', 'P2:134.0', 'P2:135.0', 'P2:136.0', 'P2:137.0',
    'P2:138.0', 'P2:139.0', 'P2:14.0', 'P2:140.0', 'P2:141.0',
    'P2:142.0', 'P2:143.0', 'P2:144.0', 'P2:145.0', 'P2:146.0',
    'P2:147.0', 'P2:148.0', 'P2:149.0', 'P2:15.0', 'P2:150.0',
    'P2:151.0', 'P2:152.0', 'P2:153.0', 'P2:154.0', 'P2:155.0',
    'P2:156.0', 'P2:157.0', 'P2:158.0', 'P2:159.0', 'P2:16.0',
    'P2:160.0', 'P2:161.0', 'P2:162.0', 'P2:164.0', 'P2:165.0',
    'P2:166.0', 'P2:167.0', 'P2:168.0', 'P2:169.0', 'P2:17.0',
    'P2:170.0', 'P2:171.0', 'P2:172.0', 'P2:173.0', 'P2:174.0',
    'P2:175.0', 'P2:176.0', 'P2:177.0', 'P2:178.0', 'P2:179.0',
    'P2:18.0', 'P2:180.0', 'P2:181.0', 'P2:182.0', 'P2:183.0',
    'P2:184.0', 'P2:185.0', 'P2:186.0', 'P2:187.0', 'P2:188.0',
    'P2:189.0', 'P2:19.0', 'P2:190.0', 'P2:191.0', 'P2:192.0',
    'P2:193.0', 'P2:194.0', 'P2:195.0', 'P2:196.0', 'P2:197.0',
    'P2:198.0', 'P2:199.0', 'P2:2.0', 'P2:20.0', 'P2:200.0',
    'P2:201.0', 'P2:202.0', 'P2:203.0', 'P2:204.0', 'P2:205.0',
    'P2:206.0', 'P2:207.0', 'P2:208.0', 'P2:209.0', 'P2:21.0',
    'P2:210.0', 'P2:211.0', 'P2:212.0', 'P2:213.0', 'P2:214.0',
    'P2:215.0', 'P2:216.0', 'P2:217.0', 'P2:218.0', 'P2:219.0',
    'P2:220.0', 'P2:221.0', 'P2:222.0', 'P2:223.0', 'P2:224.0',
    'P2:225.0', 'P2:226.0', 'P2:227.0', 'P2:228.0', 'P2:229.0',
    'P2:23.0', 'P2:230.0', 'P2:231.0', 'P2:232.0', 'P2:233.0',
    'P2:234.0', 'P2:235.0', 'P2:236.0', 'P2:237.0', 'P2:238.0',
    'P2:239.0', 'P2:24.0', 'P2:240.0', 'P2:241.0', 'P2:243.0',
    'P2:244.0', 'P2:245.0', 'P2:246.0', 'P2:247.0', 'P2:248.0',
    'P2:249.0', 'P2:25.0', 'P2:250.0', 'P2:251.0', 'P2:252.0',
    'P2:253.0', 'P2:255.0', 'P2:256.0', 'P2:257.0', 'P2:258.0',
    'P2:259.0', 'P2:26.0', 'P2:260.0', 'P2:261.0', 'P2:262.0',
    'P2:263.0', 'P2:27.0', 'P2:28.0', 'P2:3.0', 'P2:30.0', 'P2:32.0',
    'P2:33.0', 'P2:34.0', 'P2:35.0', 'P2:38.0', 'P2:39.0', 'P2:4.0',
    'P2:40.0', 'P2:41.0', 'P2:42.0', 'P2:43.0', 'P2:44.0', 'P2:45.0',
    'P2:46.0', 'P2:47.0', 'P2:48.0', 'P2:49.0', 'P2:5.0', 'P2:50.0',
    'P2:51.0', 'P2:52.0', 'P2:53.0', 'P2:54.0', 'P2:55.0', 'P2:56.0',
    'P2:57.0', 'P2:58.0', 'P2:59.0', 'P2:6.0', 'P2:60.0', 'P2:61.0',
    'P2:62.0', 'P2:63.0', 'P2:64.0', 'P2:65.0', 'P2:66.0', 'P2:67.0',
    'P2:69.0', 'P2:7.0', 'P2:70.0', 'P2:71.0', 'P2:72.0', 'P2:73.0',
    'P2:74.0', 'P2:75.0', 'P2:76.0', 'P2:77.0', 'P2:78.0', 'P2:79.0',
    'P2:8.0', 'P2:80.0', 'P2:81.0', 'P2:82.0', 'P2:83.0', 'P2:85.0',
    'P2:86.0', 'P2:88.0', 'P2:89.0', 'P2:9.0', 'P2:90.0', 'P2:91.0',
    'P2:92.0', 'P2:93.0', 'P2:94.0', 'P2:95.0', 'P2:96.0', 'P2:97.0',
    'P2:98.0', 'P2:99.0'
]].sum()

# Get Online Data

In [24]:
# Read in Data
online = pd.read_csv('data/online.csv')

In [25]:
online_sessions_df = get_online_sessions_df(online.copy())

In [26]:
customer_online_counts = online_sessions_df.groupby('custno', as_index=False)[[
    'E1:-1.0', 'E1:1.0', 'E1:10.0', 'E1:11.0',
       'E1:2.0', 'E1:4.0', 'E1:5.0', 'E1:6.0', 'E1:7.0', 'E1:8.0',
       'E1:9.0', 'E2:1', 'E2:10', 'E2:2', 'E2:3', 'E2:4', 'E2:5', 'E2:6',
       'E2:7', 'E2:8', 'E2:9', 'Cat:1', 'Cat:2', 'Cat:3'
]].sum()

# Merge Data Together

In [21]:
total_customer_orders.head()

,P1:1.0,P1:2.0,P1:3.0,P1:4.0,P1:5.0,P1:7.0,P2:-1.0,P2:-7.0,P2:10.0,P2:100.0,P2:101.0,P2:102.0,P2:103.0,P2:104.0,P2:105.0,P2:106.0,P2:107.0,P2:108.0,P2:109.0,P2:11.0,P2:110.0,P2:111.0,P2:112.0,P2:113.0,P2:114.0,P2:115.0,P2:116.0,P2:117.0,P2:118.0,P2:119.0,P2:12.0,P2:120.0,P2:121.0,P2:122.0,P2:123.0,P2:124.0,P2:125.0,P2:126.0,P2:127.0,P2:128.0,P2:129.0,P2:13.0,P2:130.0,P2:131.0,P2:132.0,P2:133.0,P2:134.0,P2:135.0,P2:136.0,P2:137.0,...,P2:51.0,P2:52.0,P2:53.0,P2:54.0,P2:55.0,P2:56.0,P2:57.0,P2:58.0,P2:59.0,P2:6.0,P2:60.0,P2:61.0,P2:62.0,P2:63.0,P2:64.0,P2:65.0,P2:66.0,P2:67.0,P2:69.0,P2:7.0,P2:70.0,P2:71.0,P2:72.0,P2:73.0,P2:74.0,P2:75.0,P2:76.0,P2:77.0,P2:78.0,P2:79.0,P2:8.0,P2:80.0,P2:81.0,P2:82.0,P2:83.0,P2:85.0,P2:86.0,P2:88.0,P2:89.0,P2:9.0,P2:90.0,P2:91.0,P2:92.0,P2:93.0,P2:94.0,P2:95.0,P2:96.0,P2:97.0,P2:98.0,P2:99.0
custno,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,5,0,0,0,0,1,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,7,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
all_customer_engagement = customer_online_counts.merge(
    total_customer_orders, on='custno', how='outer'
)

In [40]:
all_customer_engagement = all_customer_engagement.fillna(0)

In [36]:
# Quality Check
assert all_customer_engagement['custno'].shape[0] == len(all_customer_engagement['custno'].unique())
assert customer_online_counts['custno'].shape[0] == len(customer_online_counts['custno'].unique())
assert total_customer_orders['custno'].shape[0] == len(total_customer_orders['custno'].unique())

assert 0 == len(
    set(all_customer_engagement['custno']) 
    - set(customer_online_counts['custno']).union(set(total_customer_orders['custno']))
)

In [43]:
all_customer_engagement.to_pickle('data/AE_Clustering/all_customer_engagement.pkl')